<a href="https://colab.research.google.com/github/pamunarr/Chroma/blob/main/Chroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chroma - Caso de uso

**ARDA II - NoSQL**

*Andra-Iulia Cebuc, Pablo Munarriz Senosiain*

En este cuaderno mostramos un caso de uso del SGBD vectorial [Chroma](https://docs.trychroma.com/). Además de poner en práctica lo aprendido, nuestro objetivo es relacionarlo con la asignatura Aprendizaje Profundo, de forma que podamos ver el funcionamiento de Chroma en el contexto para el que fue diseñado.

## Librerías

En primer lugar, debemos instalar las librerías necesarias. Para usar Chroma basta con instalar `chromadb`, pero también utilizaremos `datasets` y `cohere`. Una vez instaladas, reiniciamos el kernel para vaciar la memoria RAM.

In [ ]:
!pip install chromadb
!pip install datasets -Uqq
!pip install -U cohere

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
import chromadb
from datasets import load_dataset
import cohere

# Dataset

Vamos a trabajar con datos de la Wikipedia, concretamente, vamos a descargarnos documentos en euskera y gallego. Para ello recurrimos a HuggingFace, en particular a [este dataset](https://huggingface.co/datasets/Cohere/wikipedia-2023-11-embed-multilingual-v3), que contiene una copia de Wikipedia del 01/11/2023 en más de 300 idiomas. Empezamos descargando los documentos en euskera.

In [2]:
wikipedia_eu = load_dataset("Cohere/wikipedia-2023-11-embed-multilingual-v3", "eu", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Vamos a echar un vistazo al esquema de los datos.

In [3]:
wikipedia_eu

Dataset({
    features: ['_id', 'url', 'title', 'text', 'emb'],
    num_rows: 1327579
})

Tenemos cinco características:
 - `_id`: un identificador,
 - `url`: dirección URL del documento,
 - `title`: título del documento,
 - `text`: contenido del documento,
 - `emb`: un embedding del documento.

Ya que disponemos de la URL y de un embedding, vamos a aprovechar para no guardar el texto como tal, y solo trabajar con los embeddings. La propia URL nos sirve para saber a qué documento hace mención el embedding en cuestión.

Cabe mencionar que, según dicen en la página del dataset, los embeddings los han creado mediante el modelo de embedding [Cohere Embed V3](https://txt.cohere.com/introducing-embed-v3/), más concrétamente, mediante el modelo [Cohere-embed-multilingual-v3.0](https://huggingface.co/Cohere/Cohere-embed-multilingual-v3.0).

En definitiva, en nuestra colección guardaremos, para cada documento, el identificador, el embedding, el título como documento (así tendremos mayor flexibilidad en las consultas) y, como metadatos, la URL y el idioma.

Por suspuesto, antes de hablar de colecciones, debemos obtener el cliente. En este caso, trabajaremos en disco (cliente persistente), y así podremos reiniciar el kernel sin perder los datos.


In [4]:
client = chromadb.PersistentClient(path = "./chromadb")
client.heartbeat() # de esta forma comprobamos la conexión

1712319156046676243

Ahora debemos crear la colección. Para ello deberemos también conseguir la función de embedding concreta, puesto que queremos hacer consultas mediante texto y no mediante embeddings. El siguiente código se puede entender revisando los links [uno](https://docs.trychroma.com/embeddings#custom-embedding-functions) y [dos](https://huggingface.co/Cohere/Cohere-embed-multilingual-v3.0).

In [5]:
class MyEmbeddingFunction(chromadb.EmbeddingFunction):
  def __call__(self, input: chromadb.Documents) -> chromadb.Embeddings:
    co = cohere.Client("") # API key de Cohere
    embeddings = co.embed(
        texts = input ,
        input_type = "search_query" , # solo usaremos el embedding function para las querys
        model = "embed-multilingual-v3.0"
    ).embeddings
    return embeddings

emb_fn = MyEmbeddingFunction()

collection = client.create_collection(
  name = "wikipedia" , # nombre de la colección
  embedding_function = emb_fn , # función de embedding
  metadata = {"hnsw:space" : "cosine"} # función de distancia entre embeddings
)

Ahora vamos a añadir datos a la colección. Como el dataset es bastante grande, vamos a meterlo poco a poco. Para ello primero creamos una función que cumpla con el cometido.

In [6]:
def add_dataset(ds , collection , idioma , L = 500):
  # L indica el número de documentos a añadir a la vez
  N = ds.num_rows # número total de documentos

  n_0 = 0 # posición del primer elemento a añadir
  while n_0 < N:
    n_1 = min(n_0 + L , N)

    # Nos quedamos con los documentos a añadir y escribimos también el idioma
    ds_0 = ds.select(range(n_0 , n_1)).add_column("lang" , [idioma] * (n_1 - n_0))

    # Añadimos los embeddings, junto a los metadatos y los identificadores
    collection.add(
        embeddings = ds_0["emb"] ,
        documents = ds_0["title"] ,
        metadatas = ds_0.select_columns(["url" , "lang"]).to_list() ,
        ids = ds_0["_id"]
    )

    n_0 = n_1 # actualizamos n_0
    print("Porcentaje completado: %.2f" % (100 * n_0 / N))

Ya estamos listos para añadir los embeddings.

In [7]:
add_dataset(wikipedia_eu.remove_columns("text") , collection , "eu")

Porcentaje completado: 0.04
Porcentaje completado: 0.08
Porcentaje completado: 0.11
Porcentaje completado: 0.15
Porcentaje completado: 0.19
Porcentaje completado: 0.23
Porcentaje completado: 0.26
Porcentaje completado: 0.30
Porcentaje completado: 0.34
Porcentaje completado: 0.38
Porcentaje completado: 0.41
Porcentaje completado: 0.45
Porcentaje completado: 0.49
Porcentaje completado: 0.53
Porcentaje completado: 0.56
Porcentaje completado: 0.60
Porcentaje completado: 0.64
Porcentaje completado: 0.68
Porcentaje completado: 0.72
Porcentaje completado: 0.75
Porcentaje completado: 0.79
Porcentaje completado: 0.83
Porcentaje completado: 0.87
Porcentaje completado: 0.90
Porcentaje completado: 0.94
Porcentaje completado: 0.98
Porcentaje completado: 1.02
Porcentaje completado: 1.05
Porcentaje completado: 1.09
Porcentaje completado: 1.13
Porcentaje completado: 1.17
Porcentaje completado: 1.21
Porcentaje completado: 1.24
Porcentaje completado: 1.28
Porcentaje completado: 1.32
Porcentaje completad

Comprobamos que todo está correcto. Para ello tenemos las siguientes funciones:

In [8]:
print("Numero de documentos:" , collection.count())
primero = collection.peek(1) # extraemos el primer elemento
print("ID:" , primero["ids"][0])
print("Embedding:" , str(primero["embeddings"][0][:2])[:-1] , "..." , str(primero["embeddings"][0][-2:])[1:])
print("Metadatos:" , primero["metadatas"][0])
print("Título:" , primero["documents"][0])

Numero de documentos: 1327579
ID: 20231101.eu_1000001_0
Embedding: [0.0100860595703125, -0.0005135536193847656 ... 0.037078857421875, -0.05780029296875]
Metadatos: {'lang': 'eu', 'url': 'https://eu.wikipedia.org/wiki/Monterreyko%20Teknologikoa'}
Título: Monterreyko Teknologikoa


Ya tenemos los documentos en euskara en la base de datos, ahora vamos a hacer lo mismo para los documentos en gallego.

Vamos a aprovechar a reiniciar el kernel, así que tendremos que reescribir el código ya que perderemos las variables.

In [1]:
import chromadb
from datasets import load_dataset
import cohere

In [2]:
wikipedia_gl = load_dataset("Cohere/wikipedia-2023-11-embed-multilingual-v3", "gl", split="train").remove_columns("text")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
client = chromadb.PersistentClient(path = "./chromadb")

class MyEmbeddingFunction(chromadb.EmbeddingFunction):
  def __call__(self, input: chromadb.Documents) -> chromadb.Embeddings:
    co = cohere.Client("") # API key de Cohere
    embeddings = co.embed(
        texts = input ,
        input_type = "search_query" , # solo usaremos el embedding function para las querys
        model = "embed-multilingual-v3.0"
    ).embeddings
    return embeddings

emb_fn = MyEmbeddingFunction()

collection = client.get_collection(
  name = "wikipedia" ,
  embedding_function = emb_fn ,
)

def add_dataset(ds , collection , idioma , L = 500):
  # L indica el número de documentos a añadir a la vez
  N = ds.num_rows # número total de documentos

  n_0 = 0 # posición del primer elemento a añadir
  while n_0 < N:
    n_1 = min(n_0 + L , N)

    # Nos quedamos con los documentos a añadir y escribimos también el idioma
    ds_0 = ds.select(range(n_0 , n_1)).add_column("lang" , [idioma] * (n_1 - n_0))

    # Añadimos los embeddings, junto a los metadatos y los identificadores
    collection.add(
        embeddings = ds_0["emb"] ,
        documents = ds_0["title"] ,
        metadatas = ds_0.select_columns(["url" , "lang"]).to_list() ,
        ids = ds_0["_id"]
    )

    n_0 = n_1 # actualizamos n_0
    print("Porcentaje completado: %.2f" % (100 * n_0 / N))

add_dataset(wikipedia_gl , collection , "gl" , L = 800)

Porcentaje completado: 0.08
Porcentaje completado: 0.15
Porcentaje completado: 0.23
Porcentaje completado: 0.30
Porcentaje completado: 0.38
Porcentaje completado: 0.45
Porcentaje completado: 0.53
Porcentaje completado: 0.61
Porcentaje completado: 0.68
Porcentaje completado: 0.76
Porcentaje completado: 0.83
Porcentaje completado: 0.91
Porcentaje completado: 0.98
Porcentaje completado: 1.06
Porcentaje completado: 1.14
Porcentaje completado: 1.21
Porcentaje completado: 1.29
Porcentaje completado: 1.36
Porcentaje completado: 1.44
Porcentaje completado: 1.51
Porcentaje completado: 1.59
Porcentaje completado: 1.67
Porcentaje completado: 1.74
Porcentaje completado: 1.82
Porcentaje completado: 1.89
Porcentaje completado: 1.97
Porcentaje completado: 2.04
Porcentaje completado: 2.12
Porcentaje completado: 2.19
Porcentaje completado: 2.27
Porcentaje completado: 2.35
Porcentaje completado: 2.42
Porcentaje completado: 2.50
Porcentaje completado: 2.57
Porcentaje completado: 2.65
Porcentaje completad

In [3]:
collection.count()

2384569

Ya están todos los datos cargados. Vamos a reiniciar de nuevo el kernel.

## Consultas

Ya hemos terminado de cargar documentos. Ahora nos interesa consultar la colección. Trataremos de mostrar las distintas herramientas que tenemos a nuestra disposición.

En primer lugar, vamos simplemente a filtrar embeddings como una base de datos al uso. Por ejemplo, vamos a quedarnos con los URL de los documentos en euskera cuyo título contenga la palabra "Lizarra". Para esto tenemos el método `get`.

In [1]:
import chromadb
from datasets import load_dataset
import cohere

ModuleNotFoundError: No module named 'chromadb'

In [2]:
client = chromadb.PersistentClient(path = "./chromadb")

class MyEmbeddingFunction(chromadb.EmbeddingFunction):
  def __call__(self, input: chromadb.Documents) -> chromadb.Embeddings:
    co = cohere.Client("") # API key de Cohere
    embeddings = co.embed(
        texts = input ,
        input_type = "search_query" , # solo usaremos el embedding function para las querys
        model = "embed-multilingual-v3.0"
    ).embeddings
    return embeddings

emb_fn = MyEmbeddingFunction()

collection = client.get_collection(
  name = "wikipedia" ,
  embedding_function = emb_fn ,
)

In [5]:
respuesta = collection.get(
    include = ["metadatas"] ,
    where = {
        "lang" : "eu"
    } ,
    where_document = {"$contains" : "Lizarra"}
)


In [6]:
{met["url"] for met in respuesta["metadatas"]}

{'https://eu.wikipedia.org/wiki/Amaia%20Lizarralde',
 'https://eu.wikipedia.org/wiki/Ander%20Lizarralde%20Jimeno',
 'https://eu.wikipedia.org/wiki/Antonio%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Apolonia%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Cris%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Diego%20Lizarrakoa',
 'https://eu.wikipedia.org/wiki/Done%20Mikel%20eliza%20%28Lizarra%29',
 'https://eu.wikipedia.org/wiki/Elene%20Lizarralde',
 'https://eu.wikipedia.org/wiki/Eneko%20Lizarralde',
 'https://eu.wikipedia.org/wiki/Foruen%20plaza%20%28Lizarra%29',
 'https://eu.wikipedia.org/wiki/Gabriel%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Garazi%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Garestik%20Lizarrara%20%28Donejakue%20bidea%29',
 'https://eu.wikipedia.org/wiki/Gerardo%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Gobernadorearen%20Jauregia%20%28Lizarra%29',
 'https://eu.wikipedia.org/wiki/Hilobi%20Santuaren%20eliza%20%28Lizarra%29',
 'https://eu.wikipedia.org/wiki/Ir

In [ ]:
collection.query(
    query_texts = ["Biderketa"] ,
    n_results = 5 ,
    include = ["metadatas"]
)

{'ids': [['20231101.eu_334769_0',
   '20231101.eu_334767_0',
   '20231101.eu_850803_37',
   '20231101.eu_294750_0',
   '20231101.eu_667096_0']],
 'distances': None,
 'metadatas': [[{'lang': 'eu',
    'title': 'Bikote (pertsonen arteko harremana)',
    'url': 'https://eu.wikipedia.org/wiki/Bikote%20%28pertsonen%20arteko%20harremana%29'},
   {'lang': 'eu',
    'title': 'Bikote (argipena)',
    'url': 'https://eu.wikipedia.org/wiki/Bikote%20%28argipena%29'},
   {'lang': 'eu',
    'title': 'Arbitraje (zuzenbidea)',
    'url': 'https://eu.wikipedia.org/wiki/Arbitraje%20%28zuzenbidea%29'},
   {'lang': 'eu',
    'title': 'Besarkada',
    'url': 'https://eu.wikipedia.org/wiki/Besarkada'},
   {'lang': 'eu',
    'title': 'Bizarra mozte',
    'url': 'https://eu.wikipedia.org/wiki/Bizarra%20mozte'}]],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None}